In [ ]:
!pip install spotipy --upgrade

import numpy as np
import pandas as pd
import os
import requests

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from difflib import SequenceMatcher

In [ ]:
SPOTIFY_CLIENT_ID = 'key'
SPOTIFY_CLIENT_SECRET = 'key'

auth_manager = SpotifyClientCredentials(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
request_limit = 2480

spotify = spotipy.Spotify(auth_manager=auth_manager)

results = spotify.search(q='%20track:repentless%20artist:slayer%20', type='track,album', limit=10)
print(results['tracks'])
print(results['albums'])

# for artist in results["artists"]["items"]:
#   print(artist)
#   print(artist['name'])
#   print(artist["genres"])
#   print(artist["external_urls"])
# # items = results['artists']['items']
# # if len(items) > 0:
# #     artist = items[0]
# #     print(artist['name'], artist['images'][0]['url'])

In [3]:
all_songs = pd.read_csv('./tcc_ceds_music_full.csv')

In [ ]:
start_idx=0 #pop
artist_permissible_similarity = 0.95
track_permissible_similarity = 0.85
#country = 7042
#blues = 12487
#jazz = 17091
#reggae = 20936
#rock = 23434
#hip-hop = 27468
#28371 - last

In [ ]:
for idx,(artist,track,year) in enumerate(zip(all_songs["artist_name"][start_idx:],all_songs["track_name"][start_idx:],all_songs["release_date"][start_idx:])):
  artist = artist.strip().lower()
  track = track.strip().lower()

  print(f"Artist: {artist}; Track: {track}")
  
  request_succeeded = False
  while not request_succeeded:
    try:
      search_result = spotify.search(q=f'%20track:{track}%20artist:{artist}%20year:{year-3}-{year+3}%20', type='track', limit=50)
      request_succeeded = True
    except:
      pass

  if search_result['tracks']['total'] == 0:
    print("No matching items found")
    continue

  cover_url = ''

  #picks the best matching response
  max_artist_match_ratio = 0
  max_track_match_ratio = 0
  artist_diff_epsilon = 0.05

  for search_item in search_result['tracks']['items']:
    examined_artist_name = search_item['artists'][0]['name'].strip().lower()
    examined_track_name = search_item['name'].strip().lower()

    print(f"Examined artist: {examined_artist_name}; Examined track: {examined_track_name}")
    
    artist_match_ratio = np.max([SequenceMatcher(None, examined_artist_name, artist).ratio(), SequenceMatcher(None, artist, examined_artist_name).ratio() ])
    track_match_ratio = np.max([SequenceMatcher(None, examined_track_name, track).ratio(), SequenceMatcher(None, track, examined_track_name).ratio() ])

    update_cover_url = False
    if artist_match_ratio > max_artist_match_ratio:
      max_artist_match_ratio = artist_match_ratio
      update_cover_url = True
    
    if abs(artist_match_ratio - max_artist_match_ratio) <= artist_diff_epsilon and track_match_ratio > max_track_match_ratio:
      max_artist_match_ratio = np.max([artist_match_ratio, max_artist_match_ratio])
      max_track_match_ratio = track_match_ratio
      update_cover_url = True

    if update_cover_url:
      album_item = search_item['album']
      if len(album_item['images']) > 0:
        cover_url = album_item['images'][0]['url']

  if cover_url == '':
    print(f"No suitable cover found for idx {idx+start_idx}")
    continue
  
  print(f"Eventual URL for idx {idx+start_idx}: {cover_url}")
  print(f"Best artist match: {max_artist_match_ratio}; Best track match: {max_track_match_ratio}")
  downloaded_cover = requests.get(cover_url)

  # Save the image to the specified path
  with open(f"./retrieved_covers/{idx+start_idx}__{artist.replace('/','_')}__{track.replace('/','_')}__.jpeg", "wb") as f:
      f.write(downloaded_cover.content)





